In [18]:
%load_ext autoreload
%autoreload 2

In [19]:
import numpy as np
import sklearn
import pickle
from sklearn.ensemble import RandomForestClassifier
from preprocess import *
from sklearn.preprocessing import MinMaxScaler    
from sklearn.metrics import accuracy_score

from bbsQt.model import kinect_utils as ku 
from bbsQt.model import rec_utils as ru

In [20]:
# To do: 0이 아닌 점들에 대해서만.. 
# 좌-우 중 하나만 0이면 반대편도 같이 제외
def shift_to_zero(skeleton, nframe=2, njoints=30):
    early_frame_x = skeleton[:nframe*njoints:2]
    early_frame_y = skeleton[1:nframe*njoints:2]

    ix_nz = np.nonzero(early_frame_x)
    iy_nz = np.nonzero(early_frame_y)

    mean_x = np.mean(early_frame_x[ix_nz])
    mean_y = np.mean(early_frame_y[iy_nz])
    
    skeleton[::2] -= mean_x 
    skeleton[1::2] -= mean_y 
    return skeleton

from bbsQt.model import BBS_pp_utils as bbpp
# 키 normalize 
## 어깨 - 팔꿈치 / 허벅지 / 종아리 
common_joints = bbpp.COMMON_JOINT

xy_joint_inds = dict([(name,i) for i, name in enumerate([prefix+cj for cj in common_joints for prefix in ["x", "y"]])])

def joint_length(joint, j1:str, j2:str):
    """measure lentgh of a joint from j1 to j2"""
    x1 = joint[xy_joint_inds["x"+j1]]
    x2 = joint[xy_joint_inds["x"+j2]]
    y1 = joint[xy_joint_inds["y"+j1]]
    y2 = joint[xy_joint_inds["y"+j2]]
    return np.sqrt((x1-x2)**2 + (y1-y2)**2)
    

def arms_and_legs(skeleton, topk=2):
    """
    get representative length of arms and legs
    """
    lls = []
    rls = []
    las = []
    ras = []
    for sk in skeleton.reshape(-1, 30):
        lls.append(joint_length(sk, 'l_hip', 'l_knee'))
        rls.append(joint_length(sk, 'r_hip', 'r_knee'))
        las.append(joint_length(sk, 'l_elbow', 'l_shoulder'))
        ras.append(joint_length(sk, 'r_elbow', 'r_shoulder')) 

    out = []    
    for arr in (lls, rls, las, ras):
        out.append(arr[np.argsort(arr)[-topk]])
    return out


PAIRS = {'larm':('l_elbow', 'l_shoulder'),
         'rarm':('r_elbow', 'r_shoulder'), 
         'lleg':('l_hip', 'l_knee'),
         'rleg':('r_hip', 'r_knee'),
         'body':('neck', 'pelvis')}

def measure_lengths(skeleton, topk=2):
    """
    get representative lengths of joints
    """
    
    out = []
    for pair in PAIRS:
        lengths = []
        for sk in skeleton.reshape(-1, 30):
            lengths.append(joint_length(sk, *PAIRS[pair]))
            
        out.append((pair,lengths[np.argsort(lengths)[-topk]]))
            
    return dict(out)

# 스켈레톤이 아주 stochastic하기 때문에 5장 중 median으로 smoothing

In [35]:
def load_all_npy(base_dir, action, nframe, frame_skip = 20, feature_ready=False, cam = "e", FPS=15):
    ### BBS data set
    main_list = f"whoismain/whoismain_{cam}_txt/main_list_{cam}_{action}.txt"
    npy_list = merge_main_npy(base_dir+main_list, prefix=base_dir + f"npy_{cam}/")

    scene = np.load(npy_list[0]['npy'])
    data = []
    label=[]
    durations=[]

    for i, fn_npy in enumerate(npy_list):
        if fn_npy['main'] in [0,1]:
            try:
            #if True:
                scene = np.load(fn_npy['npy'])
                for j in range(frame_skip):
                    sub = smoothed_frame_N(scene, nframe=nframe, shift=j)

                    # feature 기록
                    rav_sub = ravel_rec(sub)
                    
                    data.append(rav_sub)
                    label.append(fn_npy['score'])

                    # Calculate duration of the clip
                    durations.append(max(sub['frame'])/FPS)

            except:
                print("Main person was guessed, but not actually detected... ", fn_npy['npy'])


    features = [ff for i in range(nframe) for ff in sub.dtype.names if ff not in "frame" ]
    features = np.array(features)

    data = np.stack(data)
    label = np.array(label)
    print(action, data.shape)
    return data, label, durations

In [36]:
def std_data(data):
    for joint in data:
        joint = shift_to_zero(joint)

    for dd in data:
        body = measure_lengths(dd)
        dd /= body['body'] 

In [23]:
def run_train(data, label, action, cam, ntrees=20, max_depth = 7):
    X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(data, 
                                                                                label, 
                                                                                test_size=0.7, 
                                                                                stratify=label)

    scaler = MinMaxScaler()
    # Use whole data to impose stricter boundary condition
    X_total = np.concatenate((X_test, X_train))
    scaler.fit(X_total)

    X_train_normed = scaler.transform(X_train)
    X_test_normed = scaler.transform(X_test)

    model = RandomForestClassifier(n_estimators=ntrees, max_depth=max_depth)
    model.fit(X_train_normed, Y_train)
    
    pickle.dump(scaler, open(f"scaler_{action}_{cam}.pickle", 'wb'))
    np.savetxt(f"scaler_{action}_{cam}_scale.txt", scaler.scale_, fmt='%.10f')
    np.savetxt(f"scaler_{action}_{cam}_min.txt", scaler.min_, fmt='%.10f')    

    pred = model.predict(X_test_normed)
    print("정확도 :{0:.3f}".format(accuracy_score(Y_test, pred)))

    pickle.dump(model, open(f"trained_model_{action}_{cam}.pickle", "wb"))

    data_to_save = {"train_x":X_train_normed, 
                     "train_y":Y_train,
                     'valid_x':X_test_normed,
                     'valid_y':Y_test}
    pickle.dump(data_to_save, open(f"BBS_dataset_{action}_{cam}.pickle", "wb"))

In [61]:
nframe = 8
CAM_LIST= {1: "e",
           2: "e",
           3: "a",
           4: "e",
           5: "e",
           6: "e",
           7: "e",
           8: "a",
           9: "a",
           10:"e",
           11:"e",
           12:"e",
           13:"a",
           14:"e"}

# 3, 8, 9, 13
for action in range(1,15):
    cam = CAM_LIST[action]
    data, label, duration = load_all_npy("/home/hoseung/Work/data/BBS/", action, nframe, cam=cam)
    
    std_data(data)
    
    # 훈련 돌리기 전에 duration 합치기 
    data = np.column_stack((data, duration))
    
    
    ## Remove any nan
    bad_row = np.unique(np.where(np.isnan(data))[0])
    data = np.delete(data, bad_row, axis=0)

    
    label = np.array([ll for i,ll in enumerate(label) if not i in bad_row])
    
    
    
    run_train(data, label, action, cam)

1 (8400, 240)
정확도 :0.924
2 (8380, 240)
정확도 :0.932
Main person was guessed, but not actually detected...  /home/hoseung/Work/data/BBS/npy_a/3/4/045/a_045_3_4_1.npy
Main person was guessed, but not actually detected...  /home/hoseung/Work/data/BBS/npy_a/3/4/062/a_062_3_4_1.npy
Main person was guessed, but not actually detected...  /home/hoseung/Work/data/BBS/npy_a/3/4/063/a_063_3_4_1.npy
Main person was guessed, but not actually detected...  /home/hoseung/Work/data/BBS/npy_a/3/2/067/a_067_3_2_1.npy
Main person was guessed, but not actually detected...  /home/hoseung/Work/data/BBS/npy_a/3/1/069/a_069_3_1_1.npy
Main person was guessed, but not actually detected...  /home/hoseung/Work/data/BBS/npy_a/3/0/070/a_070_3_0_1.npy
Main person was guessed, but not actually detected...  /home/hoseung/Work/data/BBS/npy_a/3/0/071/a_071_3_0_1.npy
Main person was guessed, but not actually detected...  /home/hoseung/Work/data/BBS/npy_a/3/1/082/a_082_3_1_1.npy
Main person was guessed, but not actually dete

/home/hoseung/anaconda3/envs/newbbs/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/hoseung/anaconda3/envs/newbbs/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


정확도 :0.958
4 (8380, 240)
정확도 :0.901
5 (8360, 240)
정확도 :0.935
6 (8340, 240)
정확도 :0.920
7 (8380, 240)
정확도 :0.919
Main person was guessed, but not actually detected...  /home/hoseung/Work/data/BBS/npy_a/8/3/045/a_045_8_3_1.npy
8 (8200, 240)
정확도 :0.904
Main person was guessed, but not actually detected...  /home/hoseung/Work/data/BBS/npy_a/9/3/003/a_003_9_3_1.npy
Main person was guessed, but not actually detected...  /home/hoseung/Work/data/BBS/npy_a/9/3/044/a_044_9_3_1.npy
Main person was guessed, but not actually detected...  /home/hoseung/Work/data/BBS/npy_a/9/3/059/a_059_9_3_1.npy
Main person was guessed, but not actually detected...  /home/hoseung/Work/data/BBS/npy_a/9/3/086/a_086_9_3_1.npy
Main person was guessed, but not actually detected...  /home/hoseung/Work/data/BBS/npy_a/9/1/132/a_132_9_1_1.npy
9 (8240, 240)


/home/hoseung/anaconda3/envs/newbbs/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/hoseung/anaconda3/envs/newbbs/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


정확도 :0.937
10 (8360, 240)
정확도 :0.868
11 (8360, 240)
정확도 :0.909
12 (8360, 240)
정확도 :0.957
Main person was guessed, but not actually detected...  /home/hoseung/Work/data/BBS/npy_a/13/1/003/a_003_13_1_1.npy
Main person was guessed, but not actually detected...  /home/hoseung/Work/data/BBS/npy_a/13/3/044/a_044_13_3_0.npy
Main person was guessed, but not actually detected...  /home/hoseung/Work/data/BBS/npy_a/13/0/045/a_045_13_0_0.npy
Main person was guessed, but not actually detected...  /home/hoseung/Work/data/BBS/npy_a/13/0/054/a_054_13_0_0.npy
Main person was guessed, but not actually detected...  /home/hoseung/Work/data/BBS/npy_a/13/0/055/a_055_13_0_0.npy
Main person was guessed, but not actually detected...  /home/hoseung/Work/data/BBS/npy_a/13/2/056/a_056_13_2_0.npy
Main person was guessed, but not actually detected...  /home/hoseung/Work/data/BBS/npy_a/13/1/057/a_057_13_1_1.npy
Main person was guessed, but not actually detected...  /home/hoseung/Work/data/BBS/npy_a/13/0/058/a_058_13

In [57]:
data.shape

(7393, 240)

In [58]:
len(duration)

7420